In [1]:
# !pip install tensorflow -qq

In [2]:
import tensorflow_datasets as tfds

In [3]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, f1_score
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete9ZOXTH/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete9ZOXTH/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete9ZOXTH/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_dataset, test_dataset = dataset["train"], dataset["test"]

In [6]:
tokenizer = info.features["text"].encoder

In [7]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [8]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)

In [9]:
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))

In [10]:
test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(tokenizer.vocab_size, 64, mask_zero=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
model.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('sigmoid'))

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience = 3, verbose=1, restore_best_weights=True,)
ls = tf.keras.callbacks.LearningRateScheduler(scheduler)

h = model.fit(train_dataset,epochs=20, validation_data=test_dataset, callbacks=[es])

Epoch 1/20
391/391 [==============================] - 178s 406ms/step - loss: 0.6437 - accuracy: 0.7558 - val_loss: 0.4183 - val_accuracy: 0.8467
Epoch 2/20
391/391 [==============================] - 131s 334ms/step - loss: 0.3373 - accuracy: 0.8915 - val_loss: 0.3517 - val_accuracy: 0.8655
Epoch 3/20
391/391 [==============================] - 115s 294ms/step - loss: 0.2616 - accuracy: 0.9235 - val_loss: 0.3297 - val_accuracy: 0.8698
Epoch 4/20
391/391 [==============================] - 106s 271ms/step - loss: 0.2068 - accuracy: 0.9434 - val_loss: 0.4431 - val_accuracy: 0.7986
Epoch 5/20
391/391 [==============================] - 102s 260ms/step - loss: 0.1607 - accuracy: 0.9612 - val_loss: 0.3618 - val_accuracy: 0.8551
Epoch 6/20
391/391 [==============================] - 97s 247ms/step - loss: 0.1306 - accuracy: 0.9700 - val_loss: 0.3675 - val_accuracy: 0.8550
Epoch 6: early stopping


In [13]:
y_test = test_dataset.map(lambda x, y: y)
y_test = tf.concat(list(y_test), axis = 0)
y_test = tf.reshape(y_test, [-1]).numpy()

y_pred = model.predict(test_dataset)
y_pred = np.where(y_pred >= 0.5, 1, 0)

print(classification_report(y_test, y_pred))

391/391 [==============================] - 30s 66ms/step
              precision    recall  f1-score   support

           0       0.85      0.89      0.87     12500
           1       0.89      0.85      0.87     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



In [14]:
test_loss, test_acc = model.evaluate(test_dataset)
f1 = f1_score(y_test, y_pred)
print('Test loss: ', test_loss)
print('Test Accuracy: ', test_acc)
print('Test F1 score: ', f1)

391/391 [==============================] - 26s 66ms/step - loss: 0.3297 - accuracy: 0.8698
Test loss:  0.3296515643596649
Test Accuracy:  0.8698400259017944
Test F1 score:  0.866683054736152
